In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Setup ---
options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(), options=options)

driver.get("https://vimandvigor.trainerize.com/app/client/17606051/classFinder")

wait = WebDriverWait(driver, 20)

# --- Fill Email ---
email_input = wait.until(
    EC.presence_of_element_located((By.ID, "emailInput"))
)
email_input.clear()
email_input.send_keys("mari_katada@hotmail.com")

# --- Fill Password ---
password_input = wait.until(
    EC.presence_of_element_located((By.ID, "passInput"))
)
password_input.clear()
password_input.send_keys("1577Villarita")

# --- Wait until Sign In button is enabled & click ---
sign_in_button = wait.until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='signIn-button']"))
)
sign_in_button.click()

wait = WebDriverWait(driver, 5)

# Wait for the "Find a class" button to be clickable
find_class_btn = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "a[data-testid='find-class']")
    )
)

find_class_btn.click()

In [30]:
driver.quit()

In [34]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# ======================
# SETUP
# ======================
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--headless=new") 

driver = webdriver.Chrome(
    service=Service(),
    options=options
)

wait = WebDriverWait(driver, 20)


# ======================
# LOGIN
# ======================
driver.get("https://vimandvigor.trainerize.com/app/client/17606051/classFinder")

email_input = wait.until(
    EC.presence_of_element_located((By.ID, "emailInput"))
)
email_input.clear()
email_input.send_keys("mari_katada@hotmail.com")

password_input = wait.until(
    EC.presence_of_element_located((By.ID, "passInput"))
)
password_input.clear()
password_input.send_keys("1577Villarita")

sign_in_button = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "button[data-testid='signIn-button']")
    )
)
sign_in_button.click()


# ======================
# NAVIGATE TO CLASS FINDER
# ======================
find_class_btn = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "a[data-testid='find-class']")
    )
)
find_class_btn.click()


# ======================
# WAIT FOR CLASS GRID
# ======================
class_grid = wait.until(
    EC.presence_of_element_located((By.ID, "classFinderGrid"))
)

time.sleep(2)  # allow initial render


# ======================
# SCROLL TO LOAD ALL DAYS
# ======================
last_height = 0

for _ in range(10):  # safety cap
    driver.execute_script(
        "arguments[0].scrollTop = arguments[0].scrollHeight",
        class_grid
    )
    time.sleep(1)

    new_height = driver.execute_script(
        "return arguments[0].scrollHeight",
        class_grid
    )

    if new_height == last_height:
        break
    last_height = new_height


# ======================
# EXTRACT CLASSES
# ======================
rows = []

day_headers = class_grid.find_elements(By.TAG_NAME, "h3")

for header in day_headers:
    day_label = header.text.strip()

    # content block right after day header
    container = header.find_element(
        By.XPATH, "following-sibling::div[1]"
    )

    # skip empty days
    if "nullContainer" in container.get_attribute("class"):
        continue

    try:
        table = container.find_element(By.TAG_NAME, "table")
        trs = table.find_elements(By.TAG_NAME, "tr")

        for tr in trs:
            tds = tr.find_elements(By.TAG_NAME, "td")
            if not tds:
                continue

            rows.append({
                "day": day_label,
                "class_name": tds[0].text.strip(),
                "time": tds[1].text.strip() if len(tds) > 1 else None,
                "trainer": tds[2].text.strip() if len(tds) > 2 else None,
                "location": tds[3].text.strip() if len(tds) > 3 else None
            })

    except Exception:
        continue


# ======================
# CREATE DATAFRAME
# ======================
df = pd.DataFrame(rows)

print(df)
print(f"\nTotal classes found: {len(df)}")

# ======================
# OPTIONAL: SAVE
# ======================
# df.to_csv("vim_and_vigor_classes.csv", index=False)

# driver.quit()


                 day                     class_name time            trainer  \
0           Tomorrow    8:00 am - 9:00 am\n(60 min)    1               Step   
1           Tomorrow   9:15 am - 10:15 am\n(60 min)    1              U-JAM   
2           Tomorrow  10:30 am - 11:30 am\n(60 min)    1       LITE AND FIT   
3   27 Dec, Saturday    8:00 am - 9:00 am\n(60 min)    1               Yoga   
4   27 Dec, Saturday    9:00 am - 9:45 am\n(45 min)    1              Cycle   
5   27 Dec, Saturday   9:30 am - 10:30 am\n(60 min)    1              Zumba   
6   27 Dec, Saturday  10:45 am - 11:45 am\n(60 min)    1          Body Pump   
7     28 Dec, Sunday    8:30 am - 9:30 am\n(60 min)    1               Yoga   
8     28 Dec, Sunday   9:00 am - 10:00 am\n(60 min)    1              Cycle   
9     28 Dec, Sunday   12:00 pm - 1:00 pm\n(60 min)    1              Zumba   
10    29 Dec, Monday    8:00 am - 9:00 am\n(60 min)    1              Barre   
11    29 Dec, Monday   9:15 am - 10:15 am\n(60 min) 

In [35]:
df

,day,class_name,time,trainer,location
0,Tomorrow,8:00 am - 9:00 am\n(60 min),1,Step,6064 Camp\nGroup Exercise Room
1,Tomorrow,9:15 am - 10:15 am\n(60 min),1,U-JAM,6064 Camp\nGroup Exercise Room
2,Tomorrow,10:30 am - 11:30 am\n(60 min),1,LITE AND FIT,6064 Camp\nGroup Exercise Room
3,"27 Dec, Saturday",8:00 am - 9:00 am\n(60 min),1,Yoga,6064 Camp\nGroup Exercise Room
4,"27 Dec, Saturday",9:00 am - 9:45 am\n(45 min),1,Cycle,6064 Camp\nCycling Room
5,"27 Dec, Saturday",9:30 am - 10:30 am\n(60 min),1,Zumba,6064 Camp\nGroup Exercise Room
6,"27 Dec, Saturday",10:45 am - 11:45 am\n(60 min),1,Body Pump,6064 Camp\nGroup Exercise Room
7,"28 Dec, Sunday",8:30 am - 9:30 am\n(60 min),1,Yoga,6064 Camp\nGroup Exercise Room
8,"28 Dec, Sunday",9:00 am - 10:00 am\n(60 min),1,Cycle,6064 Camp\nCycling Room
9,"28 Dec, Sunday",12:00 pm - 1:00 pm\n(60 min),1,Zumba,6064 Camp\nGroup Exercise Room
